# Prompt Optimisation Using MIPRO

This method, MIPRO, rewrites the prompt. So start with a simple vague prompt and get it re-written to something that fits. 

Use Case:  
Create Meal/Dish classes with a prompt that is vague to start off with (i.e. "Create a meal"). Then put in a constraint that it only knows about during the optimisation phase (i.e. "Only select Fish dishes"). Then optimise the prompt and see if it changes it to cater for this constraint.

# Setup
Note: Update OPENAI_API_KEY in env settings or MySettings class, see repo's README.

In [90]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

Getting keys from environment variables


# Creat LLM Clients

In [ ]:
# Smaller LLM, this is the one that we are trying to optimize for, the prompts are going to be tweaked
# to get the best out of this model
lm_gpt35 = dspy.LM('gpt-3.5-turbo', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
#dspy.configure(lm=lm_gpt35)

lm_gpt41mini = dspy.LM('gpt-4.1-mini', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)

# # Larger LLM, this is the one that we are going to use to optimize the prompts
# # It will be the helper/teach/AI Judge to assist in the optimization process
lm_gpt4 = dspy.LM('gpt-4.1', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4)

# Create Domain

In [92]:
# Create domain classes

from pydantic import BaseModel
from typing import Literal
    
class MealDecider(dspy.Signature):
    """Suggest a dish for todays meal"""
    suggested_meal: str = dspy.InputField()

    name: str = dspy.OutputField()
    # description: str = dspy.OutputField()
    dish_type: Literal["Fish", "Vegetarian", "Chicken"] = dspy.OutputField()

mealDeciderPredict = dspy.Predict(MealDecider)
mealDeciderPredict(suggested_meal="From the ocean")

Prediction(
    name='Grilled Salmon with Lemon and Dill',
    dish_type='Fish'
)

# Create Trainset
The ideal number of trainset examples for the DSPy MIPRO Prompt Optimizer depends on the complexity of your task and the variability in your data. However, here are some general guidelines:

Recommended Starting Point
1. Small-scale tasks or prototyping: Start with **20–50 examples**.
1. Medium complexity tasks: Use **100–500 examples**.
1. High variability or complex tasks: Consider **1,000+ examples**, if computationally feasible.

In [93]:

trainset = [
    dspy.Example(
        suggested_meal="Heavy dinner meal",
        name="Lamb Rogan Josh",
        #description="A rich and hearty Indian curry made with tender lamb chunks slow-cooked in a blend of aromatic spices, tomatoes, onions, and yogurt. Served with fragrant basmati rice and garlic naan, it’s a satisfying meal perfect for a heavy dinner.",
        dish_type="Lamb"
    ).with_inputs("suggested_meal")
    ,
    dspy.Example(
        suggested_meal="Breakfast suggestion",
        name="Avocado Toast with Poached Egg",
        #description="A slice of whole grain toast topped with creamy avocado, a perfectly poached egg, a sprinkle of salt and pepper, and a touch of fresh herbs. Light, nutritious, and perfect for a wholesome breakfast.",
        dish_type="Vegetarian"
    ).with_inputs("suggested_meal")
    ,
    dspy.Example(
        suggested_meal="Something light",
        name="Grilled Salmon with Asparagus",
        #description="A healthy grilled salmon fillet served with steamed asparagus and a lemon butter sauce.",
        dish_type="Fish"
    ).with_inputs("suggested_meal")    
]


# Optimization

Purpose of metric in MIPRO
The metric function in MIPRO is used to evaluate the quality of a model’s output against the expected result. It should return a boolean or score indicating whether the output is correct.

In [94]:
def validate_match(expected, actual, trace=None) -> bool:
    print()
    md("**expected**: ", expected)
    md("**actual**: ", actual)
    md("**Is Fish**: ", actual.dish_type == "Fish")
    print()
    return (actual.dish_type == "Fish")

from dspy.teleprompt import *

tp = dspy.MIPROv2(metric=validate_match, auto="medium", prompt_model=lm_gpt35, task_model=lm_gpt4)
optimized_matcher = tp.compile(mealDeciderPredict, trainset=trainset, requires_permission_to_run=False)

2025/09/03 16:45:13 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: False
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 2

2025/09/03 16:45:13 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/09/03 16:45:13 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/09/03 16:45:13 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


  0%|          | 0/1 [00:00<?, ?it/s]

**expected**: Example({'suggested_meal': 'Heavy dinner meal', 'name': 'Lamb Rogan Josh', 'dish_type': 'Lamb'}) (input_keys={'suggested_meal'})

**actual**: Prediction(  
    name='Butter Chicken with Garlic Naan',  
    dish_type='Chicken'  
)

...

**expected**: Example({'suggested_meal': 'Breakfast suggestion', 'name': 'Avocado Toast with Poached Egg', 'dish_type': 'Vegetarian'}) (input_keys={'suggested_meal'})

**actual**: Prediction(  
    name='Spinach and Feta Omelette',  
    dish_type='Vegetarian'  
)

**Is Fish**: False


Average Metric: 1.00 / 2 (50.0%): 100%|██████████| 2/2 [00:00<00:00,  2.25it/s] 

2025/09/03 16:46:17 INFO dspy.evaluate.evaluate: Average Metric: 1 / 2 (50.0%)
2025/09/03 16:46:17 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 50.0 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 11'].
2025/09/03 16:46:17 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [50.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 50.0, 0.0, 0.0, 50.0, 50.0, 100.0, 50.0, 50.0, 50.0, 0.0, 0.0, 50.0]
2025/09/03 16:46:17 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 100.0
2025/09/03 16:46:17 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/09/03 16:46:17 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 100.0!


In [95]:
optimized_matcher.save("./saved_files/", save_program=True)

# Inspect results

In [104]:

optimized_prompt = getattr(optimized_matcher, "prompt", None) \
    or getattr(optimized_matcher, "instructions", None) \
    or getattr(optimized_matcher, "template", None)

print("**Optimized Prompt:**", optimized_prompt)

**Optimized Prompt:** None


In [105]:
if hasattr(optimized_matcher, "examples"):
    print("Few-shot examples:", optimized_matcher.examples)
elif hasattr(optimized_matcher, "demos"):
    print("Few-shot demos:", optimized_matcher.demos)

Few-shot demos: []


In [106]:
print(getattr(optimized_matcher, "compiled_template", None))

None


In [99]:

# Quick peeks (if available in your version)
try:
    print("=== optimized_matcher.inspect() ===")
    print(optimized_matcher.inspect())
except Exception:
    pass

try:
    print("=== tp.inspect() ===")
    print(tp.inspect())
except Exception:
    pass


=== optimized_matcher.inspect() ===
=== tp.inspect() ===


In [100]:
from typing import Any

def _safe(obj: Any, attr: str, default=None):
    return getattr(obj, attr, default)

def _print_section(title: str, content):
    if content:
        print(f"\n===== {title} =====")
        print(content)

def print_dspy_program(program: Any):
    print(">>> Repr:", repr(program))

    # Signature + fields
    sig = _safe(program, "signature")
    _print_section("Signature", sig)
    if sig:
        # Some versions expose .instructions or a docstring
        _print_section("Signature Doc", getattr(sig, "__doc__", None))
        _print_section("Signature Instructions", _safe(sig, "instructions"))
        _print_section("Signature Inputs", _safe(sig, "inputs", _safe(sig, "input_fields", None)))
        _print_section("Signature Outputs", _safe(sig, "outputs", _safe(sig, "output_fields", None)))

    # Where different versions might store the optimized prompt/template
    for k in ("prompt", "instructions", "system_prompt", "template", "compiled_template"):
        _print_section(k, _safe(program, k))

    # Few-shot examples (these keys vary by version/teleprompter)
    examples = None
    for k in ("examples", "fewshot", "demos", "shots", "train_examples", "demo_store"):
        v = _safe(program, k)
        if v:
            _print_section(f"{k} (container)", f"type={type(v)}, len={getattr(v, '__len__', lambda: 'n/a')() if hasattr(v, '__len__') else 'n/a'}")
            examples = v
            break

    # Try to print examples if iterable
    if examples and hasattr(examples, "__iter__"):
        try:
            for i, ex in enumerate(list(examples)[:20]):  # cap to avoid huge logs
                print(f"\n--- Few-shot Example #{i+1} ---")
                if hasattr(ex, "inputs") and callable(ex.inputs):
                    print("inputs:", ex.inputs())
                elif hasattr(ex, "inputs"):
                    print("inputs:", ex.inputs)
                else:
                    print("inputs: <unknown>")

                if hasattr(ex, "outputs") and callable(ex.outputs):
                    print("outputs:", ex.outputs())
                elif hasattr(ex, "outputs"):
                    print("outputs:", ex.outputs)
                else:
                    print("outputs: <unknown>")
        except Exception as e:
            print(f"(could not iterate examples cleanly: {e})")

print("\n================= OPTIMIZED PROGRAM =================")
print_dspy_program(optimized_matcher)


================= OPTIMIZED PROGRAM =================
>>> Repr: Predict(StringSignature(suggested_meal -> name, dish_type
    instructions="You are a renowned chef hosting a high-profile event with discerning guests. Your reputation is on the line as you need to impress them with a unique and exquisite dish. Consult the language model to predict the name and dish type of the meal you should serve for today's event."
    suggested_meal = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Suggested Meal:', 'desc': '${suggested_meal}'})
    name = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Name:', 'desc': '${name}'})
    dish_type = Field(annotation=Literal['Fish', 'Vegetarian', 'Chicken'] required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Dish Type:', 'desc': '${dish_type}'})
))

===== Signature =====
StringSignature(suggested_meal -> name, dish_type
    instructions=

In [101]:
from typing import Any

def _safe(obj: Any, attr: str, default=None):
    return getattr(obj, attr, default)

def _print_section(title: str, content):
    if content:
        print(f"\n===== {title} =====")
        print(content)

def print_dspy_program(program: Any):
    print(">>> Repr:", repr(program))

    # Signature + fields
    sig = _safe(program, "signature")
    _print_section("Signature", sig)
    if sig:
        # Some versions expose .instructions or a docstring
        _print_section("Signature Doc", getattr(sig, "__doc__", None))
        _print_section("Signature Instructions", _safe(sig, "instructions"))
        _print_section("Signature Inputs", _safe(sig, "inputs", _safe(sig, "input_fields", None)))
        _print_section("Signature Outputs", _safe(sig, "outputs", _safe(sig, "output_fields", None)))

    # Where different versions might store the optimized prompt/template
    for k in ("prompt", "instructions", "system_prompt", "template", "compiled_template"):
        _print_section(k, _safe(program, k))

    # Few-shot examples (these keys vary by version/teleprompter)
    examples = None
    for k in ("examples", "fewshot", "demos", "shots", "train_examples", "demo_store"):
        v = _safe(program, k)
        if v:
            _print_section(f"{k} (container)", f"type={type(v)}, len={getattr(v, '__len__', lambda: 'n/a')() if hasattr(v, '__len__') else 'n/a'}")
            examples = v
            break

    # Try to print examples if iterable
    if examples and hasattr(examples, "__iter__"):
        try:
            for i, ex in enumerate(list(examples)[:20]):  # cap to avoid huge logs
                print(f"\n--- Few-shot Example #{i+1} ---")
                if hasattr(ex, "inputs") and callable(ex.inputs):
                    print("inputs:", ex.inputs())
                elif hasattr(ex, "inputs"):
                    print("inputs:", ex.inputs)
                else:
                    print("inputs: <unknown>")

                if hasattr(ex, "outputs") and callable(ex.outputs):
                    print("outputs:", ex.outputs())
                elif hasattr(ex, "outputs"):
                    print("outputs:", ex.outputs)
                else:
                    print("outputs: <unknown>")
        except Exception as e:
            print(f"(could not iterate examples cleanly: {e})")

print("\n================= OPTIMIZED PROGRAM =================")
print_dspy_program(optimized_matcher)


================= OPTIMIZED PROGRAM =================
>>> Repr: Predict(StringSignature(suggested_meal -> name, dish_type
    instructions="You are a renowned chef hosting a high-profile event with discerning guests. Your reputation is on the line as you need to impress them with a unique and exquisite dish. Consult the language model to predict the name and dish type of the meal you should serve for today's event."
    suggested_meal = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Suggested Meal:', 'desc': '${suggested_meal}'})
    name = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Name:', 'desc': '${name}'})
    dish_type = Field(annotation=Literal['Fish', 'Vegetarian', 'Chicken'] required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Dish Type:', 'desc': '${dish_type}'})
))

===== Signature =====
StringSignature(suggested_meal -> name, dish_type
    instructions=

In [102]:
def print_teleprompter_history(tp_obj):
    # These attributes differ by DSPy version; we print whatever exists.
    for attr in ("history", "trials", "search_trace", "report", "best_program", "best_score", "best_state"):
        v = getattr(tp_obj, attr, None)
        if v is not None:
            print(f"\n===== Teleprompter: {attr} =====")
            print(v)

print("\n================= TELEPROMPTER HISTORY =================")
print_teleprompter_history(tp)


================= TELEPROMPTER HISTORY =================


In [103]:
import json

def extract_artifacts(program: Any):
    # Best-effort extraction across versions
    system_prompt = (
        _safe(program, "prompt") or
        _safe(program, "instructions") or
        _safe(program, "system_prompt") or
        _safe(program, "template") or
        _safe(program, "compiled_template")
    )
    sig = _safe(program, "signature")
    sig_text = None
    if sig:
        sig_text = {
            "doc": getattr(sig, "__doc__", None),
            "inputs": _safe(sig, "inputs", _safe(sig, "input_fields", None)),
            "outputs": _safe(sig, "outputs", _safe(sig, "output_fields", None)),
            "raw": repr(sig),
        }

    # Try to collect few-shots
    examples = None
    for k in ("examples", "fewshot", "demos", "shots", "train_examples", "demo_store"):
        v = _safe(program, k)
        if v:
            examples = v
            break

    few_shots = []
    if examples and hasattr(examples, "__iter__"):
        for ex in examples:
            try:
                few_shots.append({
                    "inputs": ex.inputs() if hasattr(ex, "inputs") and callable(ex.inputs) else getattr(ex, "inputs", None),
                    "outputs": ex.outputs() if hasattr(ex, "outputs") and callable(ex.outputs) else getattr(ex, "outputs", None),
                })
            except Exception:
                pass

    return {
        "system_prompt_or_template": system_prompt,
        "signature": sig_text,
        "few_shot_examples": few_shots,
    }

artifacts = extract_artifacts(optimized_matcher)

with open("mealdecider_artifacts.json", "w", encoding="utf-8") as f:
    json.dump(artifacts, f, ensure_ascii=False, indent=2)

print("Wrote:", "mealdecider_artifacts.json")

TypeError: Object of type FieldInfo is not JSON serializable